In [2]:
import os
import pandas as pd

In [2]:
data_dir = r"C:\Users\diogo\OneDrive\Documents\Dev\Creating\Projetos\Python\Python Dashboard Equal\excel_data"

cols_by_archive = {}
for archive in os.listdir(data_dir):
    if archive[:5] == "clean":
        df = pd.read_excel(f"{data_dir}\\{archive}")
        cols_by_archive[archive] = df.columns.to_list()
        
print(cols_by_archive)

{'clean_dim_familia_produtos.xlsx': ['codigo_familia', 'descricaofamilia'], 'clean_dim_produtos.xlsx': ['codigo_produto', 'descricaoproduto', 'codigo_familia', 'custo_produto_unitario'], 'clean_dim_vendedor.xlsx': ['codigo_vendedor', 'nome_vendedor'], 'clean_fato_vendas.xlsx': ['codigo_contrato', 'data_venda', 'codigo_produto', 'filial_venda', 'quantidade', 'valor_monetario_total', 'valor_desconto', 'codigo_vendedor', 'codigo_cliente']}


In [9]:
data_dir = r"C:\Users\diogo\OneDrive\Documents\Dev\Creating\Projetos\Python\Python Dashboard Equal\excel_data"
df_fato_vendas = pd.read_excel(f"{data_dir}\\clean_fato_vendas.xlsx")
df_dim_vendedor = pd.read_excel(f"{data_dir}\\clean_dim_vendedor.xlsx")
df_dim_produtos = pd.read_excel(f"{data_dir}\\clean_dim_produtos.xlsx")
df_dim_familia_produtos = pd.read_excel(f"{data_dir}\\clean_dim_familia_produtos.xlsx")

In [10]:
df_timed_stats = pd.merge(df_fato_vendas, df_dim_produtos[["codigo_familia", "descricaoproduto", "codigo_produto"]], on="codigo_produto", how="inner")
display(df_timed_stats.head())

,codigo_contrato,data_venda,codigo_produto,filial_venda,quantidade,valor_monetario_total,valor_desconto,codigo_vendedor,codigo_cliente,codigo_familia,descricaoproduto
0,2280272,2021-03-29,2792100,17,250,29975.0,0.0,548,574535,DLN,Versatile Office Supplies Maroon #213
1,2290067,2021-04-07,2792100,14,20,2398.0,0.0,395,114034,DLN,Versatile Office Supplies Maroon #213
2,2294242,2021-04-08,2792100,14,20,2398.0,0.0,395,114034,DLN,Versatile Office Supplies Maroon #213
3,2344558,2021-05-24,2792100,16,10,1199.0,0.0,474,125763,DLN,Versatile Office Supplies Maroon #213
4,2368174,2021-06-16,2792100,16,10,1199.0,0.0,474,125763,DLN,Versatile Office Supplies Maroon #213


In [11]:
df_timed_stats = pd.merge(df_timed_stats, df_dim_familia_produtos[["descricaofamilia", "codigo_familia"]], on="codigo_familia", how="inner")
display(df_timed_stats)

,codigo_contrato,data_venda,codigo_produto,filial_venda,quantidade,valor_monetario_total,valor_desconto,codigo_vendedor,codigo_cliente,codigo_familia,descricaoproduto,descricaofamilia
0,2280272,2021-03-29,2792100,17,250,29975.0,0.0,548,574535,DLN,Versatile Office Supplies Maroon #213,Office Supplies
1,2290067,2021-04-07,2792100,14,20,2398.0,0.0,395,114034,DLN,Versatile Office Supplies Maroon #213,Office Supplies
2,2294242,2021-04-08,2792100,14,20,2398.0,0.0,395,114034,DLN,Versatile Office Supplies Maroon #213,Office Supplies
3,2344558,2021-05-24,2792100,16,10,1199.0,0.0,474,125763,DLN,Versatile Office Supplies Maroon #213,Office Supplies
4,2368174,2021-06-16,2792100,16,10,1199.0,0.0,474,125763,DLN,Versatile Office Supplies Maroon #213,Office Supplies
...,...,...,...,...,...,...,...,...,...,...,...,...
519726,2521809,2021-11-19,2886106,1,1,79.8,0.0,478,752620,CAMF,Versatile Hand Tools Purple #191,Hand Tools
519727,2523120,2021-11-22,2886106,16,1,79.8,0.0,378,58399,CAMF,Versatile Hand Tools Purple #191,Hand Tools
519728,2539521,2021-12-08,2886106,14,1,79.8,0.0,395,123183,CAMF,Versatile Hand Tools Purple #191,Hand Tools
519729,2567755,2022-01-08,2886106,14,1,79.8,0.0,441,143289,CAMF,Versatile Hand Tools Purple #191,Hand Tools


In [12]:
df_timed_stats["data_venda"] = pd.to_datetime(df_timed_stats["data_venda"]).reset_index(drop=True)
display(df_timed_stats)

,codigo_contrato,data_venda,codigo_produto,filial_venda,quantidade,valor_monetario_total,valor_desconto,codigo_vendedor,codigo_cliente,codigo_familia,descricaoproduto,descricaofamilia
0,2280272,2021-03-29,2792100,17,250,29975.0,0.0,548,574535,DLN,Versatile Office Supplies Maroon #213,Office Supplies
1,2290067,2021-04-07,2792100,14,20,2398.0,0.0,395,114034,DLN,Versatile Office Supplies Maroon #213,Office Supplies
2,2294242,2021-04-08,2792100,14,20,2398.0,0.0,395,114034,DLN,Versatile Office Supplies Maroon #213,Office Supplies
3,2344558,2021-05-24,2792100,16,10,1199.0,0.0,474,125763,DLN,Versatile Office Supplies Maroon #213,Office Supplies
4,2368174,2021-06-16,2792100,16,10,1199.0,0.0,474,125763,DLN,Versatile Office Supplies Maroon #213,Office Supplies
...,...,...,...,...,...,...,...,...,...,...,...,...
519726,2521809,2021-11-19,2886106,1,1,79.8,0.0,478,752620,CAMF,Versatile Hand Tools Purple #191,Hand Tools
519727,2523120,2021-11-22,2886106,16,1,79.8,0.0,378,58399,CAMF,Versatile Hand Tools Purple #191,Hand Tools
519728,2539521,2021-12-08,2886106,14,1,79.8,0.0,395,123183,CAMF,Versatile Hand Tools Purple #191,Hand Tools
519729,2567755,2022-01-08,2886106,14,1,79.8,0.0,441,143289,CAMF,Versatile Hand Tools Purple #191,Hand Tools


In [13]:
df_timed_stats["mes"] = df_timed_stats["data_venda"].dt.month
df_timed_stats["ano"] = df_timed_stats["data_venda"].dt.year
display(df_timed_stats)

,codigo_contrato,data_venda,codigo_produto,filial_venda,quantidade,valor_monetario_total,valor_desconto,codigo_vendedor,codigo_cliente,codigo_familia,descricaoproduto,descricaofamilia,mes,ano
0,2280272,2021-03-29,2792100,17,250,29975.0,0.0,548,574535,DLN,Versatile Office Supplies Maroon #213,Office Supplies,3,2021
1,2290067,2021-04-07,2792100,14,20,2398.0,0.0,395,114034,DLN,Versatile Office Supplies Maroon #213,Office Supplies,4,2021
2,2294242,2021-04-08,2792100,14,20,2398.0,0.0,395,114034,DLN,Versatile Office Supplies Maroon #213,Office Supplies,4,2021
3,2344558,2021-05-24,2792100,16,10,1199.0,0.0,474,125763,DLN,Versatile Office Supplies Maroon #213,Office Supplies,5,2021
4,2368174,2021-06-16,2792100,16,10,1199.0,0.0,474,125763,DLN,Versatile Office Supplies Maroon #213,Office Supplies,6,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519726,2521809,2021-11-19,2886106,1,1,79.8,0.0,478,752620,CAMF,Versatile Hand Tools Purple #191,Hand Tools,11,2021
519727,2523120,2021-11-22,2886106,16,1,79.8,0.0,378,58399,CAMF,Versatile Hand Tools Purple #191,Hand Tools,11,2021
519728,2539521,2021-12-08,2886106,14,1,79.8,0.0,395,123183,CAMF,Versatile Hand Tools Purple #191,Hand Tools,12,2021
519729,2567755,2022-01-08,2886106,14,1,79.8,0.0,441,143289,CAMF,Versatile Hand Tools Purple #191,Hand Tools,1,2022


In [14]:
df_timed_stats_per_month = df_timed_stats.groupby(["codigo_familia", "mes", "ano"])["valor_monetario_total"].sum().reset_index()
df_timed_stats_per_month = pd.merge(df_timed_stats_per_month, df_dim_familia_produtos[["descricaofamilia", "codigo_familia"]], on="codigo_familia", how="inner")
display(df_timed_stats_per_month)

,codigo_familia,mes,ano,valor_monetario_total,descricaofamilia
0,0,1,2022,19231.18,Ebooks
1,0,1,2023,21907.41,Ebooks
2,0,1,2024,19042.83,Ebooks
3,0,2,2021,7138.53,Ebooks
4,0,2,2022,15422.08,Ebooks
...,...,...,...,...,...
2670,SCF,11,2022,12679.06,Building Materials
2671,SCF,11,2023,14417.49,Building Materials
2672,SCF,12,2021,21759.79,Building Materials
2673,SCF,12,2022,28734.98,Building Materials


In [43]:
df_timed_stats_per_month.to_excel(f"{data_dir}\\df_fam_prod_venda_timed.xlsx", index=False)